# Fixed Point Grover's algorithm

We have seen that the probability of measuring the marked state $a$ after $t$ steps of Grover's algorithm is $\sin^2((2t+1)\theta)$. 
This means that if we perform too many steps we decrease the probability of measuring $a$.
It turns out that a modification of Grover's algorithm allows us to avoid this, and produces an algorithm which has $a$ as a fixed point, so that the more we run the algorithm, the more accurate result we get.
In this coursework, you will implement such an algorithm and verify its properties.

## Setup

Make sure you have installed Qiskit following [https://docs.quantum.ibm.com/start/install](https://docs.quantum.ibm.com/start/install) and install also the AerSimulator with `pip install qiskit-aer`

Run these cells to import the needed modules

In [ ]:
# Built-in modules
import math
import warnings
warnings.filterwarnings("ignore")

# Imports from Qiskit
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram

from qiskit_aer import AerSimulator

## Generalised V

We define an operator $S(\beta)$ that acts on $n+1$ qubits, such that
$$
S(\beta)|x\rangle|0\rangle = e^{-i\beta/2}V(\beta)|x\rangle|0\rangle\,,\quad 
V(\beta) = \exp(i\beta |a\rangle\langle a|)= I - (1-e^{i\beta})|a\rangle\langle a|\,.
$$
Here $\beta$ is a given angle and note that $V(\pm \pi)$ coincides with $V$ of the lecture. 

### Exercise 1: Implement $S(\beta)$ (40 points)
Create a function which takes as inputs $\beta,a$ and returns a Qiskit circuit for $S(\beta)$ by implementing the following circuit:

<img src="s_beta.png" alt="S(beta) circuit" style="height: 300px; width:400px;"/>

where $U=U_f$ is as usual the oracle such that 
$$
U_f|a\rangle|y\rangle=|a\rangle|y\oplus 1\rangle
\,,\quad 
U_f|a_\perp\rangle|y\rangle=|a_\perp\rangle|y\rangle
$$
for any $\langle a|a_\perp\rangle=0$ and
$$
Z_\beta
=
\exp\left(-i\frac{\beta}{2}Z\right)
$$
with $Z$ the Pauli matrix.

In [ ]:
def create_s(marked_state, beta):
    """Implement S(\beta)

    Parameters:
        marked_state (str): Marked state of oracle (a)
        beta (float): angle

    Returns:
        QuantumCircuit: Quantum circuit representing S(\beta)
    """
    # ...

## Generalised inversion by the mean

We define the generalised inversion by the mean as
$$
W(\alpha)
=
e^{i\alpha/4}
(
I - (1-e^{-i\alpha})|\phi\rangle\langle\phi|
)
$$
with 
$$
|\phi\rangle
=
A|0\rangle^{\otimes n}
\,,\quad 
A = H^{\otimes n}
.
$$
For $\alpha=\pm$ it coincides with the inversion by the mean seen in the lecture up to a phase.

### Exercise 2: Implement $W(\alpha)$ (60 points)
Create a function which takes as inputs $\alpha$ and returns a Qiskit circuit for $W(\alpha)$ by implementing the following circuit:

<img src="w_alpha.png" alt="W(alpha) circuit" style="height: 300px; width:600px;"/>

where $A=H^{\otimes n}$, and

<img src="empty_cnot.png" alt="Negative CNOT " style="height: 50px; width:300px;"/>

Hint: to implement the multi-controlled CNOT gate you can use the Qiskit method `mcx`

In [ ]:
def create_w(alpha, num_qubits):
    """Implement W(\alpha)

    Parameters:
        alpha (float): angle
        num_qubits (int): number of qubits

    Returns:
        QuantumCircuit: Quantum circuit representing W(\alpha)
    """
    # ...

## The full algorithm 

The full circuit is given by
$$
G
=
\prod_{i=1}^l W(\alpha_j) S(\beta_j) 
$$
The angles $\alpha_j, \beta_j$ can be chosen in such a way that the success probability in finding $a$ increases with $l$. We can guarantee a probability greater than $1-\delta^2$ for $\delta\in (0,1]$ if we choose $L \ge \log(2/\delta) \sqrt{N}$ and the angles as defined below, with $L=2l+1$. Note the square root scaling with $N$ as in Grover's, to which the algorithm reduces when $\delta=1$.

In [ ]:
def acot(x):
    return math.atan2(1, x)    

def cheb(i, x):
    return math.cosh(i*math.acosh(x))

def gamma(L, delta):
    return 1./cheb(1./L, 1./delta)

def alpha_fixed_point(j, l, delta):
    L = 2*l+1
    return 2*acot(math.tan(2*math.pi*j/L)*math.sqrt(1 - gamma(L, delta)**2))

def beta_fixed_point(j, l, delta):
    return -alpha_fixed_point(l-j+1, l, delta)

In [ ]:
def create_fixed_point_circuit(marked_state, num_steps, delta):
    """Implement G

    Parameters:
        marked_state (str): Marked state of oracle (a)
        num_steps (int): Number of steps of the algorithm (l)
        delta (float): Precision parameter in (0,1]

    Returns:
        QuantumCircuit: Quantum circuit G with input state preparation
    """
    # Compute the number of qubits in circuit
    num_qubits = len(marked_state)

    qc = QuantumCircuit(num_qubits + 1)

    # Prepare input state
    qc.h(list(range(num_qubits)))

    for j in range(num_steps):
        alpha = alpha_fixed_point(j + 1, num_steps, delta)
        beta = beta_fixed_point(j + 1, num_steps, delta)
        qc.compose(create_s(marked_state, beta), inplace=True)
        qc.compose(create_w(alpha, num_qubits), inplace=True)
    return qc

Now run this algorithm for $a=00101$, $\delta=0.5$ and varying number of steps. 

In [ ]:
fp_qc = create_fixed_point_circuit('00101', 8, .5)
fp_qc.measure_all()
#fp_qc.draw(output="mpl", style="iqp")

In [ ]:
result = AerSimulator().run(fp_qc, shots=1024, memory=True).result()
counts = result.get_counts()
plot_histogram(counts)

Compare with the case $\delta=1$ which corresponds to Grover's, to check the fixed point property.

In [ ]:
# Probability of measuring a in Grover's
[math.sin((2*t+1) * math.asin(1/math.sqrt(32)) )**2 for t in range(10)]

In [ ]:
grover_qc = create_fixed_point_circuit('00101', 8, 1)
grover_qc.measure_all()
result = AerSimulator().run(grover_qc, shots=1024, memory=True).result()
counts = result.get_counts()
plot_histogram(counts)

As expected 8 steps of Grover's algorithm give a much worse estimate than say 5 steps, while the fixed point algorithm does not suffer from this problem.

## References

- https://learning.quantum.ibm.com/tutorial/grovers-algorithm
- [https://arxiv.org/abs/1409.3305](https://arxiv.org/abs/1409.3305).
- See also [https://arxiv.org/abs/2105.02859](https://arxiv.org/abs/2105.02859) for a more general perspective